# Null Model
This code generates several null models either generatively or randomly changing aspects of a loaded in dataset.

In [ ]:
import pandas as pd
import networkx as nx
import random
from tqdm.notebook import tqdm 


In [13]:
# Load node list
node_file = "networks/'blues'_node_list.csv"  # Update with your actual file name
edge_file = "networks/'blues'_edge_list.csv"  # Update with your actual file name

# Read the node list
nodes_df = pd.read_csv(node_file)
node_ids = nodes_df["Node"].tolist()  # Extract node IDs

# Read the edge list
edges_df = pd.read_csv(edge_file)

# Ensure edges are stored as (min, max) for consistency
edges = {(min(u, v), max(u, v)) for u, v in zip(edges_df["Node1"], edges_df["Node2"])}

In [15]:
# Create the original graph
G = nx.Graph()
G.add_nodes_from(node_ids)
G.add_edges_from(edges)

# Get the number of edges
num_edges = len(G.edges)

# Perform edge swaps
num_swaps = 10 * num_edges  # More swaps = more randomization while keeping degree fixed
G_null = G.copy()

# Edge swapping function for undirected graphs
for _ in tqdm(range(num_swaps), desc="Swapping edges", unit=" swaps"):
    # Pick two random edges (A-B and C-D)
    (a, b), (c, d) = random.sample(list(G_null.edges), 2)

    # Ensure we're not creating self-loops or duplicate edges
    if len({a, b, c, d}) == 4:  # Ensures all nodes are unique
        new_edge1 = (min(a, d), max(a, d))
        new_edge2 = (min(c, b), max(c, b))

        if new_edge1 not in G_null.edges and new_edge2 not in G_null.edges:
            G_null.remove_edge(a, b)
            G_null.remove_edge(c, d)
            G_null.add_edge(*new_edge1)
            G_null.add_edge(*new_edge2)

# Extract new randomized edges
null_edges = sorted(G_null.edges())  # Sorting ensures consistent ordering

# Save the null model edge list
null_edges_df = pd.DataFrame(null_edges, columns=["Node1", "Node2"])
null_edges_df.to_csv("null_edges.csv", index=False)

print(f"Null model created with {len(null_edges)} edges (same as original) and saved as 'null_edges.csv'.")

Swapping edges: 100%|██████████| 60000/60000 [02:58<00:00, 335.99 swaps/s]

Null model created with 6000 edges (same as original) and saved as 'null_edges.csv'.
